<a href="https://colab.research.google.com/github/PRASANNA-416/Human-Activity-Detection/blob/main/MobileNetv3_small_with_Cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd

# Load the training CSV file
train_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Final_training_set.csv'
train_data = pd.read_csv(train_csv_path)

# Load the testing CSV file
test_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Testing_set.csv'
test_data = pd.read_csv(test_csv_path)

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Define the target image size and batch size
target_size = (224, 224)  # Adjust according to your requirements
batch_size = 64

# Create ImageDataGenerators with preprocessing options
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values between 0 and 1
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create train and test generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/FINAL_TRAIN',  # Directory containing the train images
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/test',
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Check the class indices assigned by the generator
class_indices = test_generator.class_indices
print(class_indices)


Found 17566 validated image filenames belonging to 15 classes.
Found 2520 validated image filenames belonging to 15 classes.
{'calling': 0, 'clapping': 1, 'cycling': 2, 'dancing': 3, 'drinking': 4, 'eating': 5, 'fighting': 6, 'hugging': 7, 'laughing': 8, 'listening_to_music': 9, 'running': 10, 'sitting': 11, 'sleeping': 12, 'texting': 13, 'using_laptop': 14}


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold

import pandas as pd

# Define the target image size and batch size
target_size = (224, 224)
batch_size = 64

# Load the CSV files into dataframes
train_csv = pd.read_csv('/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Final_training_set.csv')
test_csv = pd.read_csv('/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Testing_set.csv')

# Create ImageDataGenerator with preprocessing options
datagen = ImageDataGenerator(
    rescale=1.0/255.0  # Normalize pixel values between 0 and 1
)

# Define the number of folds for cross-validation
num_folds = 3

# Perform Stratified K-Fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)




In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

folder_path = "/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/FINAL_TRAIN"  # Replace with the actual path to the FINAL_TRAIN folder
csv_path = "/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Final_training_set.csv"  # Replace with the actual path to the training_set.csv file


# Read the training_set.csv file into a pandas DataFrame
df = pd.read_csv(csv_path)

# Get the list of filenames in the FINAL_TRAIN folder
file_names = os.listdir(folder_path)

# Check the count of files without corresponding labels
count = 0
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path) and file_name not in df['filename'].values:
        count += 1

print("Number of files in FINAL_TRAIN without corresponding labels:", count)





Number of files in FINAL_TRAIN without corresponding labels: 0


In [ ]:


# Read the training_set.csv file into a pandas DataFrame
df = pd.read_csv(csv_path)

# Get the list of filenames in the FINAL_TRAIN folder
file_names = os.listdir(folder_path)

# Check the count of entries without corresponding images
count = 0
for index, row in df.iterrows():
    if row['filename'] not in file_names:
        count += 1

print("Number of entries in training_set.csv without corresponding images:", count)


Number of entries in training_set.csv without corresponding images: 0


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense

def MobileNetV3Small():
    model = tf.keras.Sequential([
        # First Block
        Conv2D(16, (3, 3), strides=(2, 2), padding='same', activation='relu'),
        # Bottleneck Blocks
        Bottleneck(16, 16, 16, 1, relu6=True),
        Bottleneck(16, 72, 24, 2, relu6=False),
        Bottleneck(24, 88, 24, 1, relu6=False),
        Bottleneck(24, 96, 40, 2, relu6=True),
        Bottleneck(40, 240, 40, 1, relu6=True),
        Bottleneck(40, 240, 40, 1, relu6=True),
        Bottleneck(40, 120, 48, 1, relu6=True),
        Bottleneck(48, 144, 48, 1, relu6=True),
        Bottleneck(48, 288, 96, 2, relu6=True),
        Bottleneck(96, 576, 96, 1, relu6=True),
        Bottleneck(96, 576, 96, 1, relu6=True),
        # Last Block
        Conv2D(576, (1, 1), strides=(1, 1), padding='same', activation='relu'),
        GlobalAveragePooling2D(),
        Dense(15, activation='softmax')
    ])
    return model

def Bottleneck(input_filters, expansion, output_filters, stride, relu6):
    model = tf.keras.Sequential([
        Conv2D(expansion, (1, 1), strides=(1, 1), padding='same', activation='relu'),
        DepthwiseConv2D((3, 3), strides=(stride, stride), padding='same', activation='relu' if relu6 else None),
        Conv2D(output_filters, (1, 1), strides=(1, 1), padding='same', activation='relu')
    ])
    return model


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, DepthwiseConv2D, GlobalAveragePooling2D, Dense

def EfficientNetLite3():
    model = tf.keras.Sequential([
        Conv2D(40, kernel_size=3, strides=2, padding='same', input_shape=(224, 224, 3)),
        BatchNormalization(),
        Activation('swish'),

        # First block
        DepthwiseConv2D(kernel_size=3, strides=1, padding='same'),
        BatchNormalization(),
        Activation('swish'),
        Conv2D(80, kernel_size=1, strides=1, padding='same'),
        BatchNormalization(),
        Activation('swish'),

        # Second block
        DepthwiseConv2D(kernel_size=3, strides=2, padding='same'),
        BatchNormalization(),
        Activation('swish'),
        Conv2D(160, kernel_size=1, strides=1, padding='same'),
        BatchNormalization(),
        Activation('swish'),

        # Third block
        DepthwiseConv2D(kernel_size=3, strides=2, padding='same'),
        BatchNormalization(),
        Activation('swish'),
        Conv2D(320, kernel_size=1, strides=1, padding='same'),
        BatchNormalization(),
        Activation('swish'),

        GlobalAveragePooling2D(),
        Dense(15, activation='softmax')
    ])
    return model


In [ ]:
from tensorflow.keras import layers


In [ ]:
import tensorflow as tf
import tensorflow_hub as tfhub

# Load the MobileNet V3 Small model from TensorFlow Hub
model = tfhub.KerasLayer('https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5',
                         trainable=False,
                         arguments=dict(return_endpoints=True))

# Create an input layer with the desired input shape
input = tf.keras.layers.Input((224, 224, 3))

# Pass the input through the model to obtain the outputs
outputs = model(input)

# Print the names and shapes of the intermediate outputs
for k, v in sorted(outputs.items()):
    print(k, v.shape)


MobilenetV3/global_pool (None, 1, 1, 1024)
MobilenetV3/layer_1 (None, 112, 112, 16)
MobilenetV3/layer_10 (None, 7, 7, 96)
MobilenetV3/layer_10/depthwise_output (None, 7, 7, 288)
MobilenetV3/layer_10/expansion_output (None, 14, 14, 288)
MobilenetV3/layer_10/output (None, 7, 7, 96)
MobilenetV3/layer_11 (None, 7, 7, 96)
MobilenetV3/layer_11/depthwise_output (None, 7, 7, 576)
MobilenetV3/layer_11/expansion_output (None, 7, 7, 576)
MobilenetV3/layer_11/output (None, 7, 7, 96)
MobilenetV3/layer_12 (None, 7, 7, 96)
MobilenetV3/layer_12/depthwise_output (None, 7, 7, 576)
MobilenetV3/layer_12/expansion_output (None, 7, 7, 576)
MobilenetV3/layer_12/output (None, 7, 7, 96)
MobilenetV3/layer_13 (None, 7, 7, 576)
MobilenetV3/layer_14 (None, 1, 1, 576)
MobilenetV3/layer_15 (None, 1, 1, 1024)
MobilenetV3/layer_2 (None, 56, 56, 16)
MobilenetV3/layer_2/depthwise_output (None, 56, 56, 16)
MobilenetV3/layer_2/output (None, 56, 56, 16)
MobilenetV3/layer_3 (None, 28, 28, 24)
MobilenetV3/layer_3/depthwise_o

In [4]:
import tensorflow_hub as hub

efficientnet_lite3_base_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/efficientnet/lite3/classification/2",
    output_shape=[1280],
    trainable=False
)

print("Thickness of the model:", len(efficientnet_lite3_base_layer.weights))
print("{:<80} {:<20} {:<10}".format('Layer', 'Shape', 'Type'))

for i in range(len(efficientnet_lite3_base_layer.weights)):
    model_weights_raw_string = str(efficientnet_lite3_base_layer.weights[i])
    model_weights_wo_weights = model_weights_raw_string.split(", numpy", 1)[0]
    dtype = model_weights_wo_weights.split(" dtype=")[1]
    shape = model_weights_wo_weights.split(" shape=")[1].split(" dtype=")[0]
    
    print("{:<80} {:<20} {:<10}".format(efficientnet_lite3_base_layer.weights[i].name, shape, dtype))


Thickness of the model: 367
Layer                                                                            Shape                Type      
efficientnet-lite3/stem/conv2d/kernel:0                                          (3, 3, 3, 32)        float32   
efficientnet-lite3/stem/tpu_batch_normalization/gamma:0                          (32,)                float32   
efficientnet-lite3/stem/tpu_batch_normalization/beta:0                           (32,)                float32   
efficientnet-lite3/stem/tpu_batch_normalization/moving_mean:0                    (32,)                float32   
efficientnet-lite3/stem/tpu_batch_normalization/moving_variance:0                (32,)                float32   
efficientnet-lite3/blocks_0/depthwise_conv2d/depthwise_kernel:0                  (3, 3, 32, 1)        float32   
efficientnet-lite3/blocks_0/tpu_batch_normalization/gamma:0                      (32,)                float32   
efficientnet-lite3/blocks_0/tpu_batch_normalization/beta:0          

In [ ]:
# Iterate through the folds
fold_num = 0
for train_index, val_index in skf.split(train_csv, train_csv['label']):
    fold_num += 1
    print("Results for fold", fold_num)

    if train_index.max() < len(train_csv) and val_index.max() < len(train_csv):
        # Split the data into training and validation sets for this fold
        train_fold = train_csv.iloc[train_index]
        val_fold = train_csv.iloc[val_index]

        # Create train and validation generators for this fold
        train_generator = datagen.flow_from_dataframe(
            dataframe=train_fold,
            directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/FINAL_TRAIN',
            x_col='filename',  # Column name for the filenames
            y_col='label',  # Column name for the labels
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical',
            subset='training'  # Specify training subset for train_generator
        )

        val_generator = datagen.flow_from_dataframe(
            dataframe=train_fold,
            directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/FINAL_TRAIN',
            x_col='filename',  # Column name for the filenames
            y_col='label',  # Column name for the labels
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical',
            subset='validation'  # Specify validation subset for val_generator
        )

        mobilenet_model = MobileNetV3Small()
        efficientnet_model = EfficientNetLite3()

# Create an input layer to accept the images
        input_layer = Input(shape=(224, 224, 3))

# Get the output predictions from MobileNetV3 Small model
        mobilenet_output = mobilenet_model(input_layer)

# Get the output predictions from EfficientNet Lite3 model
        efficientnet_output = efficientnet_model(input_layer)

# Assign weightage to the model outputs
        weight = 0.7
        mobilenet_weighted = tf.keras.layers.Multiply()([
            mobilenet_output,
            tf.expand_dims(tf.expand_dims(weight, axis=0), axis=0)
        ])
        efficientnet_weighted = tf.keras.layers.Multiply()([
            efficientnet_output,
            tf.expand_dims(tf.expand_dims(1 - weight, axis=0), axis=0)
        ])

# Combine the weighted outputs using averaging
        ensemble_output = layers.Add()([mobilenet_weighted, efficientnet_weighted])

# Build the ensemble model
        ensemble_model = tf.keras.Model(inputs=input_layer, outputs=ensemble_output)


        ensemble_model.compile(
            loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(),
            metrics=['accuracy']
        )

        ensemble_model.fit(
            train_generator,
            epochs=12,
            validation_data=val_generator  # Use the validation data during training
        )

        # Evaluate the model on the test set
        test_generator = datagen.flow_from_dataframe(
            dataframe=test_csv,
            directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/test',
            x_col='filename',  # Column name for the filenames
            y_col='label',  # Column name for the labels
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical'
        )

        ensemble_model.evaluate(test_generator)
    else:
        print("Indices out of bounds")


Results for fold 1
Found 11710 validated image filenames belonging to 15 classes.
Found 0 validated image filenames belonging to 15 classes.
Epoch 1/12
183/183 [==============================] - 85s 336ms/step - loss: 2.5187 - accuracy: 0.1953
Epoch 2/12
183/183 [==============================] - 58s 317ms/step - loss: 2.4138 - accuracy: 0.2520
Epoch 3/12
183/183 [==============================] - 58s 318ms/step - loss: 2.3618 - accuracy: 0.2801
Epoch 4/12
183/183 [==============================] - 58s 318ms/step - loss: 2.3290 - accuracy: 0.2967
Epoch 5/12
183/183 [==============================] - 59s 319ms/step - loss: 2.3069 - accuracy: 0.3125
Epoch 6/12
 93/183 [==============>...............] - ETA: 28s - loss: 2.2851 - accuracy: 0.3256

KeyboardInterrupt: ignored

In [ ]:
ensemble_model.save_weights('Ensemble_mobileNetv3smallcrossvalidation.h5')
from google.colab import files
files.download('Ensemble_mobileNetv3smallcrossvalidation.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

import os
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [ ]:

def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [ ]:

TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [ ]:

tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(ensemble_model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

In [ ]:

convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

FileNotFoundError: ignored